## Introducción

En el siguiente proyecto trabajaré con la base de datos de una compañía móvil en la cual analizaré el comportamiento de sus clientes con el objetivo de recomendar un nuevo plan. Para realizar estas recomendaciones, desarrollaré un modelo de clasificación que escoja correctamente cual de los planes se ajusta mejor a cada usuario de acuerdo con sus características, con la mayor exactitud posible, con un umbral de 0.75. 

## 1. Incialización

In [1]:
#importar librerías
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

In [2]:
#librerías para elegir un modelo
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

## 1.1 Cargar datos

In [3]:
#cargar dataset
users_behav = pd.read_csv("datasets/users_behavior.csv")

In [4]:
#visualizar informacion general del dataset
users_behav.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
#visualizar datos
users_behav.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


## 2. Segmentar los datos para el modelo

In [6]:
#definir variables para las características y el objeto del modelo (conjunto de prueba)
features = users_behav.drop(['is_ultra'], axis=1)
target = users_behav['is_ultra']

#segmentar un conjunto de entranamiento y uno de validación 
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.20, random_state=12345)

## 3. Comparar la calidad de los modelos

In [7]:
#árbol de decisión de regresión
best_model = None
best_result = 10000
best_depth = 0

for depth in range (1,6):
    model = DecisionTreeRegressor(random_state=12345, max_depth=depth, min_samples_split=2, min_samples_leaf=1, splitter='best')
    model.fit(features_train, target_train)
    prediction_valid = model.predict(features_valid)

    result = mean_squared_error(target_valid, prediction_valid) ** 0.5

    if result < best_result:
        best_model = model
        best_result = result
        best_depth = depth


print(f"RECM del mejor modelo en el conjunto de validación (max_depth = {best_depth}): {best_result}")

print(f"Exactitud del modelo en el conjunto de validación = {model.score(features_valid, target_valid)}")
print(f"Exactitud del modelo en el conjunto de entrenamiento = {model.score(features_train, target_train)}")

RECM del mejor modelo en el conjunto de validación (max_depth = 5): 0.4015534785850737
Exactitud del modelo en el conjunto de validación = 0.239069247263892
Exactitud del modelo en el conjunto de entrenamiento = 0.331617391235569


Al probar un árbol de decisión de regresión me doy cuenta de que este modelo no alcanza el umbral de exactitud de 0.75 requerido para este proyecto, alcanzando 0.33. Y parece estar sobreajustado. Descarto este modelo por que el nivel de exactitud.

In [8]:
#RandomForest regresor
best_model = None
best_result = 10000
best_depth = 0
best_est = 0

for est in range(10, 50, 100):
    for depth in range(1, 11):
        model = RandomForestRegressor(random_state=12345, n_estimators=est, max_depth=depth, min_samples_leaf=1, min_samples_split=2, max_features=1)
        model.fit(features_train, target_train)
        predictions_valid = model.predict(features_valid)
        
        result = mean_squared_error(target_valid, prediction_valid) ** 0.5

        if result < best_result:
            best_model = model
            best_result = result
            best_depth = depth
            best_est = est


print(f"RECM del mejor modelo en el conjunto de validación= {best_result}, n_estimators= {best_est}, best_depth= {best_depth}")
print(f"Exactitud del modelo en el conjunto de validación = {model.score(features_valid, target_valid)}")
print(f"Exactitud del modelo en el conjunto de entrenamiento = {model.score(features_train, target_train)}")

RECM del mejor modelo en el conjunto de validación= 0.4015534785850737, n_estimators= 10, best_depth= 1
Exactitud del modelo en el conjunto de validación = 0.285532574581872
Exactitud del modelo en el conjunto de entrenamiento = 0.5279879923905082


Un bosque aleatorio de 10  árboles, con profundidad de 1 da un buen nivel de exactitud de 0.52 en el conjunto de entrenamiento y parece estar sobreajustado. 

In [9]:
#regresión logística
model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=500, penalty='l2')
model.fit(features_train, target_train)

predict_valid = model.predict(features_valid)

results = mean_squared_error(target_valid, predict_valid)

print(f"RECM del mejor modelo en el conjunto de validación = {results}")
print(f"Exactitud del modelo en el conjunto de validación = {model.score(features_valid, target_valid)}")
print(f"Exactitud del modelo en el conjunto de entrenamiento = {model.score(features_train, target_train)}")

RECM del mejor modelo en el conjunto de validación = 0.24261275272161742
Exactitud del modelo en el conjunto de validación = 0.7573872472783826
Exactitud del modelo en el conjunto de entrenamiento = 0.7467911318553092


La regresión logística tiene un error cuadrático menor en comparación con el bosque aleatorio, y una exatitud de 0.75 en el conjunto  de validación y 0.74 en el conjunto de entrenamiento. No da indicio de estar sobreajustado. Por lo tanto, este será el modelo escogido para el conjunto de prueba.

In [10]:
#regresión linear
model = LinearRegression()
model.fit(features_train, target_train)

predicts_valid = model.predict(features_valid)

result = mean_squared_error(target_valid, predicts_valid)

print(f"RECM del mejor modelo en el conjunto de validación = {result}")
print(f"Exactitud del modelo en el conjunto de validación = {model.score(features_valid, target_valid)}")
print(f"Exactitud del modelo en el conjunto de entrenamiento = {model.score(features_train, target_train)}")

RECM del mejor modelo en el conjunto de validación = 0.19642912305724222
Exactitud del modelo en el conjunto de validación = 0.07303309479416342
Exactitud del modelo en el conjunto de entrenamiento = 0.08995572905826121


La regresión lineal proporciona la menor exactitud de todos los modelos probados, por lo tanto quedará descartado.

## 4. Comprobar la calidad del modelo usando el conjunto de prueba

El modelo elegido será uno de regresión logística, el cual utilizaré con el conjunto de prueba. Elegí este modelo porque me dio la más alta exactitud en el conjunto de validación (0.75) y al compararlo con el conjunto de entrenamiento, no parece estar sobreajustado.

In [33]:
#comprobar la calidad del LogisticRegression en el conjunto de prueba
features
target

model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=500, penalty='l2')
model.fit(features_train, target_train)

test_predictions = model.predict(features)

mse = mean_squared_error(target, test_predictions)

print(f"Exactitud del modelo en el conjunto de prueba = {model.score(features, target)}")
print(f"RECM del mejor modelo en el conjunto de prueba = {mse}")


Exactitud del modelo en el conjunto de prueba = 0.7489110143123833
RECM del mejor modelo en el conjunto de prueba = 0.2510889856876167


En el conjunto de prueba, la regresión logística proporciona una exactitud del 74%, bastante parecido a la manera en que se comportó con el conjunto de validación. 

## 5. Prueba de cordura